# im2colとcol2imの理解

In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from common.util import *

In [2]:
#サンプル行列の初期化
def init_sample_matrix(filter_num=0, channel=0, height=6, width = 8):
    matrix = []

    for row in range(height):
        temp_row = []
        for col in range(width):
            elem = (row+1)*10 + col+1
            elem += channel * 100
            elem += filter_num * 1000
            temp_row.append(elem)
        matrix.append(temp_row)

    return np.array(matrix)

In [3]:
m = init_sample_matrix()
print(m)

[[11 12 13 14 15 16 17 18]
 [21 22 23 24 25 26 27 28]
 [31 32 33 34 35 36 37 38]
 [41 42 43 44 45 46 47 48]
 [51 52 53 54 55 56 57 58]
 [61 62 63 64 65 66 67 68]]


init_sample_matrixは指定したサイズの行列を生成する。各要素の十の位が行番号を示し、一の位が列番号を示す。

In [4]:
filter_size_x = 3
filter_size_y = 3
stride        = 1
pad           = 0

In [5]:
img = np.array([[m]])
print(img.shape)
col = im2col(img, filter_size_x, filter_size_y, stride, pad)
print(col)

(1, 1, 6, 8)
[[11. 12. 13. 21. 22. 23. 31. 32. 33.]
 [12. 13. 14. 22. 23. 24. 32. 33. 34.]
 [13. 14. 15. 23. 24. 25. 33. 34. 35.]
 [14. 15. 16. 24. 25. 26. 34. 35. 36.]
 [15. 16. 17. 25. 26. 27. 35. 36. 37.]
 [16. 17. 18. 26. 27. 28. 36. 37. 38.]
 [21. 22. 23. 31. 32. 33. 41. 42. 43.]
 [22. 23. 24. 32. 33. 34. 42. 43. 44.]
 [23. 24. 25. 33. 34. 35. 43. 44. 45.]
 [24. 25. 26. 34. 35. 36. 44. 45. 46.]
 [25. 26. 27. 35. 36. 37. 45. 46. 47.]
 [26. 27. 28. 36. 37. 38. 46. 47. 48.]
 [31. 32. 33. 41. 42. 43. 51. 52. 53.]
 [32. 33. 34. 42. 43. 44. 52. 53. 54.]
 [33. 34. 35. 43. 44. 45. 53. 54. 55.]
 [34. 35. 36. 44. 45. 46. 54. 55. 56.]
 [35. 36. 37. 45. 46. 47. 55. 56. 57.]
 [36. 37. 38. 46. 47. 48. 56. 57. 58.]
 [41. 42. 43. 51. 52. 53. 61. 62. 63.]
 [42. 43. 44. 52. 53. 54. 62. 63. 64.]
 [43. 44. 45. 53. 54. 55. 63. 64. 65.]
 [44. 45. 46. 54. 55. 56. 64. 65. 66.]
 [45. 46. 47. 55. 56. 57. 65. 66. 67.]
 [46. 47. 48. 56. 57. 58. 66. 67. 68.]]


参考URL1の説明に合ったように、フィルターの範囲の要素を並べて各行としていき、col展開する（参考URLの用語を流用)。
col2imも同様。ただし、こちらはimage表現に直した際に各ピクセルの値を加算していく所が特色的。

In [6]:
print(img.shape)

img_ret = col2im(col, img.shape, filter_size_x, filter_size_y, stride, pad)
print(img_ret.shape)
print(img_ret)

(1, 1, 6, 8)
(1, 1, 6, 8)
[[[[ 11.  24.  39.  42.  45.  48.  34.  18.]
   [ 42.  88. 138. 144. 150. 156. 108.  56.]
   [ 93. 192. 297. 306. 315. 324. 222. 114.]
   [123. 252. 387. 396. 405. 414. 282. 144.]
   [102. 208. 318. 324. 330. 336. 228. 116.]
   [ 61. 124. 189. 192. 195. 198. 134.  68.]]]]


## col展開でなぜ畳み込み演算が実現できるか、重みとフィルターの関係性は
### コード上での重み演算の観察

In [7]:
#common/layers.pyより関係する所のみを抜粋
class Convolution:
    #(中略)
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out


入力行列xをim2colでcol展開した後、加工したW(col_W)と行列colで行列積(col * colW)をとっている。
上記col展開の具体的な観察を踏まえ、なぜ、colとcol_W(すなわちcol_W = self.W.reshape(FN, -1).T)の行列積を取るコトが、畳み込み演算に繋がるのかを考察する。

### "self.W.reshape(FN, -1).T"って何やっているのか？

字面上は重みWをreshapeで変形したものの転置をとっている。テキストP217を見ると、入力データ、フィルター（重み）、出力データはそれぞれ以下の次元だった。

1. 入力データ：(C,H,W)。Cはチャンネル数、Hは高さ、Wは幅
2. フィルター：(FN,C,FH,FW)。FNはフィルターの数、Cはチャンネル数(入力のチャンネル数と同一値）、FH、FWはそれぞれフィルターの高さと幅
3. 出力データ：(FN,OH,OW)。FNはフィルターの数、OH、OWはそれぞれ、出力データの高さと幅。

ここで、W(フィルータ)は(FN,C,FH,FW)の4次元配列になっている。また、この時、Wに対してreshape(FN, -1)を施すとどうなるのか？
試していく。まずは、xとWを適当に用意する。なお、思考を簡略化するために、まずは、padding0、strideが1の場合で単純に考えてみる。
なお、テキストで用意されているSimpleConvNetおよび、im2colは入力データが4次元(N,C,H,W)になることを前提としている。
load_mnistのflatten引数によって、挙動が変わる。この辺を頭に入れておく必要がある。
以下、参考動作。

### 参考動作(load_mnist)

In [8]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from simple_convnet import SimpleConvNet
from common.trainer import Trainer

# データの読み込み(flatten=Falseであって、SimpleConvNet用途の場合)
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

print(x_train.shape)
print(x_train[0].shape)

# データの読み込み(flatten=Trueであって、その他用途の場合)
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True)

print(x_train.shape)
print(x_train[0].shape)

(60000, 1, 28, 28)
(1, 28, 28)
(60000, 784)
(784,)


In [9]:
pad = 0
stride = 1
#xの用意(mnistはチャンネル数1のようなので、このテストもそれに合わせる)
print("=== preparing of x===")
H = 4
W = 5
x1 = init_sample_matrix(filter_num = 1, channel=1, height=H, width=W)
x2 = init_sample_matrix(filter_num = 2, channel=1, height=H, width=W)
x = np.array([[x1],[x2]])
print(x.shape)
print(x)
#Wの用意(mnistはチャンネル数1のようなので、このテストもそれに合わせる)
print("=== preparing of W===")
FH=3
FW=3
w1 = init_sample_matrix(filter_num = 1, channel=1, height=FH, width=FW)
w2 = init_sample_matrix(filter_num = 2, channel=1, height=FH, width=FW)
#w3 = init_sample_matrix(filter_num = 2, channel=1, height=FH, width=FW)
#w4 = init_sample_matrix(filter_num = 2, channel=2, height=FH, width=FW)
#W = np.array([[w1,w2],[w3,w4]])
W = np.array([[w1],[w2]])
FN = W.shape[0]
print(W.shape)
print(W)

=== preparing of x===
(2, 1, 4, 5)
[[[[1111 1112 1113 1114 1115]
   [1121 1122 1123 1124 1125]
   [1131 1132 1133 1134 1135]
   [1141 1142 1143 1144 1145]]]


 [[[2111 2112 2113 2114 2115]
   [2121 2122 2123 2124 2125]
   [2131 2132 2133 2134 2135]
   [2141 2142 2143 2144 2145]]]]
=== preparing of W===
(2, 1, 3, 3)
[[[[1111 1112 1113]
   [1121 1122 1123]
   [1131 1132 1133]]]


 [[[2111 2112 2113]
   [2121 2122 2123]
   [2131 2132 2133]]]]


reshape(FN, -1)をかましてみる

In [12]:
print("FNの値%d" % (FN))

temp = W.reshape(FN, -1) #最初の次元だけをFNにして、あとは自動設定(-1)
print(temp.shape)  #結果として、2行18列の配列になる。
print(temp)

col_W = temp.T
print(col_W.shape)
print(col_W)
print(col_W[0,0])

FNの値2
(2, 9)
[[1111 1112 1113 1121 1122 1123 1131 1132 1133]
 [2111 2112 2113 2121 2122 2123 2131 2132 2133]]
(9, 2)
[[1111 2111]
 [1112 2112]
 [1113 2113]
 [1121 2121]
 [1122 2122]
 [1123 2123]
 [1131 2131]
 [1132 2132]
 [1133 2133]]
1111


In [11]:
#入力xをcol展開する
print("入力xのcol展開")
print(x.shape)
col = im2col(x, FW, FH, stride, pad)
print(col.shape)
print(col)

入力xのcol展開
(2, 1, 4, 5)
(12, 9)
[[1111. 1112. 1113. 1121. 1122. 1123. 1131. 1132. 1133.]
 [1112. 1113. 1114. 1122. 1123. 1124. 1132. 1133. 1134.]
 [1113. 1114. 1115. 1123. 1124. 1125. 1133. 1134. 1135.]
 [1121. 1122. 1123. 1131. 1132. 1133. 1141. 1142. 1143.]
 [1122. 1123. 1124. 1132. 1133. 1134. 1142. 1143. 1144.]
 [1123. 1124. 1125. 1133. 1134. 1135. 1143. 1144. 1145.]
 [2111. 2112. 2113. 2121. 2122. 2123. 2131. 2132. 2133.]
 [2112. 2113. 2114. 2122. 2123. 2124. 2132. 2133. 2134.]
 [2113. 2114. 2115. 2123. 2124. 2125. 2133. 2134. 2135.]
 [2121. 2122. 2123. 2131. 2132. 2133. 2141. 2142. 2143.]
 [2122. 2123. 2124. 2132. 2133. 2134. 2142. 2143. 2144.]
 [2123. 2124. 2125. 2133. 2134. 2135. 2143. 2144. 2145.]]


入力をcol展開した後は12行9列の行列になっている。画像毎に、畳込み演算対象のデータが列ベクトルとして並んでいることがよくわかる。画像の境界なしに、延べたんに展開しているのがポイントか。

# 参考URL
1. 基本的な理解(im2col/col2imの図解がわかりやすい)　https://qiita.com/t-tkd3a/items/6b17f296d61d14e12953
2. 行列積による畳込み(im2colした後の行列サイズやフィルターの解説についてわかりやすい) https://www.youtube.com/watch?v=PWPJVws7l0M